In [58]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point

In [59]:
# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords})


In [60]:
# Assuming property_gdf and victoria_gdf are already defined as per your provided code...
victoria_gdf = gpd.read_file('../data/Statistical_area_level2/victoria.geojson')

# Perform a spatial join
property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/2101387466.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")


In [62]:
PTV_metro = gpd.read_file('../data/external_SA2/PTV/PTV_metro/PTV_METRO_TRAIN_STATION.shp')
PTV_regional = gpd.read_file("../data/external_SA2/PTV/PTV_regional/PTV_REGIONAL_TRAIN_STATION.shp")

In [63]:
PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/1496773614.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)


In [66]:
station_SA2 = gpd.sjoin(PTV_station, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/2576761343.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:7844
Right CRS: EPSG:4326

  station_SA2 = gpd.sjoin(PTV_station, victoria_gdf, how="inner", op="within")


In [70]:
station_SA2 = station_SA2.drop_duplicates(subset='STOP_NAME', keep='first')

In [71]:
station_SA2.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry,index_right,SA2_CODE21,SA2_NAME21,...,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21
0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95231 -37.78118),132,206041124,Parkville,...,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,4.0491,http://linked.data.gov.au/dataset/asgsed3/SA2/...
1,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93933 -37.78813),138,206041506,North Melbourne,...,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,2.4248,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.93617 -37.79425),138,206041506,North Melbourne,...,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,2.4248,http://linked.data.gov.au/dataset/asgsed3/SA2/...
3,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.94258 -37.80741),134,206041127,West Melbourne - Industrial,...,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,6.1992,http://linked.data.gov.au/dataset/asgsed3/SA2/...
4,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99542 -37.78864),161,206071145,Clifton Hill - Alphington,...,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.2089,http://linked.data.gov.au/dataset/asgsed3/SA2/...


In [78]:
filtered = station_SA2[station_SA2['SA2_NAME21'] =='Melbourne CBD - East']

In [79]:
filtered

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry,index_right,SA2_CODE21,SA2_NAME21,...,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21


In [74]:
property_agg = pd.merge(property_SA2, station_SA2, on='SA2_CODE21', how='left')

In [75]:
property_agg.columns.values

array(['name', 'geometry_x', 'index_right_x', 'SA2_CODE21',
       'SA2_NAME21_x', 'CHG_FLAG21_x', 'CHG_LBL21_x', 'SA3_CODE21_x',
       'SA3_NAME21_x', 'SA4_CODE21_x', 'SA4_NAME21_x', 'GCC_CODE21_x',
       'GCC_NAME21_x', 'STE_CODE21_x', 'STE_NAME21_x', 'AUS_CODE21_x',
       'AUS_NAME21_x', 'AREASQKM21_x', 'LOCI_URI21_x', 'STOP_ID',
       'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'TICKETZONE', 'ROUTEUSSP',
       'geometry_y', 'index_right_y', 'SA2_NAME21_y', 'CHG_FLAG21_y',
       'CHG_LBL21_y', 'SA3_CODE21_y', 'SA3_NAME21_y', 'SA4_CODE21_y',
       'SA4_NAME21_y', 'GCC_CODE21_y', 'GCC_NAME21_y', 'STE_CODE21_y',
       'STE_NAME21_y', 'AUS_CODE21_y', 'AUS_NAME21_y', 'AREASQKM21_y',
       'LOCI_URI21_y'], dtype=object)

In [76]:
merged = property_agg[['name', 'SA2_CODE21', 'SA2_NAME21_x', 'geometry_x', 'geometry_y']]

In [77]:
merged.head()

,name,SA2_CODE21,SA2_NAME21_x,geometry_x,geometry_y
0,904/265 Exhibition Street Melbourne VIC 3000,206041503,Melbourne CBD - East,POINT (144.96912 -37.80951),None
1,2302/17 Spring Street Melbourne VIC 3000,206041503,Melbourne CBD - East,POINT (144.97440 -37.81475),None
2,11/187 Collins Street Melbourne VIC 3000,206041503,Melbourne CBD - East,POINT (144.96785 -37.81540),None
3,1605/68 La Trobe Street Melbourne VIC 3000,206041503,Melbourne CBD - East,POINT (144.96679 -37.80836),None
4,3007/35 Spring Street Melbourne VIC 3000,206041503,Melbourne CBD - East,POINT (144.97400 -37.81417),None


In [80]:
merged.rename(columns={'name': 'postcode', 'SA2_NAME21_x': 'SA2_district', 'geometry_x': 'property_geometry', 'geometry_y': 'station_geometry'}, inplace=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/876340040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'name': 'postcode', 'SA2_NAME21_x': 'SA2_district', 'geometry_x': 'property_geometry', 'geometry_y': 'station_geometry'}, inplace=True)


In [81]:
merged['postcode'] = merged['postcode'].str[-4:]

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [82]:
merged.head()

,postcode,SA2_CODE21,SA2_district,property_geometry,station_geometry
0,3000,206041503,Melbourne CBD - East,POINT (144.96912 -37.80951),None
1,3000,206041503,Melbourne CBD - East,POINT (144.97440 -37.81475),None
2,3000,206041503,Melbourne CBD - East,POINT (144.96785 -37.81540),None
3,3000,206041503,Melbourne CBD - East,POINT (144.96679 -37.80836),None
4,3000,206041503,Melbourne CBD - East,POINT (144.97400 -37.81417),None


In [83]:
filtered_df = merged[merged['SA2_district'] == 'Melbourne CBD - West']


In [84]:
filtered_df

,postcode,SA2_CODE21,SA2_district,property_geometry,station_geometry
161,3000,206041505,Melbourne CBD - West,POINT (144.96015 -37.81710),POINT (144.95566 -37.81197)
162,3000,206041505,Melbourne CBD - West,POINT (144.96015 -37.81710),POINT (144.96697 -37.81829)
163,3000,206041505,Melbourne CBD - West,POINT (144.95325 -37.81623),POINT (144.95566 -37.81197)
164,3000,206041505,Melbourne CBD - West,POINT (144.95325 -37.81623),POINT (144.96697 -37.81829)
165,3000,206041505,Melbourne CBD - West,POINT (144.96213 -37.81473),POINT (144.95566 -37.81197)
...,...,...,...,...,...
712,3000,206041505,Melbourne CBD - West,POINT (144.95364 -37.81742),POINT (144.96697 -37.81829)
713,3004,206041505,Melbourne CBD - West,POINT (144.95454 -37.81317),POINT (144.95566 -37.81197)
714,3004,206041505,Melbourne CBD - West,POINT (144.95454 -37.81317),POINT (144.96697 -37.81829)
715,3000,206041505,Melbourne CBD - West,POINT (144.95465 -37.81322),POINT (144.95566 -37.81197)


In [85]:
park = gpd.read_file('../data/external_SA2/Park.geojson')

In [86]:
park_SA2 = gpd.sjoin(park, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [87]:
property_agg1 = pd.merge(merged, park_SA2, on='SA2_CODE21', how='left')

In [88]:
merged = property_agg1[['postcode', 'SA2_CODE21','SA2_district', 'property_geometry', 'station_geometry', 'geometry']]
merged.rename(columns={'geometry': 'park_geometry'}, inplace=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/2093578972.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'park_geometry'}, inplace=True)


In [89]:
shop = gpd.read_file('../data/external_SA2/Mall, Shopping Centre & Department Store.geojson')

In [90]:
shop_SA2 = gpd.sjoin(shop, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [91]:
property_agg2 = pd.merge(merged, shop_SA2, on='SA2_CODE21', how='left')

In [92]:
merged = property_agg2[['postcode', 'SA2_CODE21','SA2_district','property_geometry', 'station_geometry', 'park_geometry', 'geometry']]
merged.rename(columns={'geometry': 'shop_geometry'}, inplace=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/853544697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'shop_geometry'}, inplace=True)


In [93]:
hospital = gpd.read_file('../data/external_SA2/Hospital.geojson')

In [94]:
hospital_SA2 = gpd.sjoin(hospital, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [95]:
property_agg3 = pd.merge(merged, hospital_SA2, on='SA2_CODE21', how='left')

In [96]:
merged = property_agg3[['postcode', 'SA2_CODE21','SA2_district','property_geometry', 'station_geometry', 'park_geometry', 'shop_geometry', 'geometry']]
merged.rename(columns={'geometry': 'hospital_geometry'}, inplace=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/3925431631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'hospital_geometry'}, inplace=True)


In [97]:
school_df = pd.read_csv("../data/external_SA2/complete_school_location.csv", encoding='latin-1')
school_df['coordinates'] = school_df.apply(lambda row: [row['Y'], row['X']], axis=1)

In [98]:
# Your property data
school_metadata = school_df

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
#school_metadata['coordinates'] = school_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in school_metadata['coordinates']]
names = school_metadata['School_Type'].tolist()

# Convert to GeoDataFrame
school = gpd.GeoDataFrame({'name': names, 'geometry': coords})

In [99]:
school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/1493813974.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")


In [100]:
property_agg4 = pd.merge(merged, school_SA2, on='SA2_CODE21', how='left')

In [101]:
merged = property_agg4[['postcode', 'SA2_CODE21','SA2_district','property_geometry', 'station_geometry', 'park_geometry', 'shop_geometry','hospital_geometry', 'geometry']]
merged.rename(columns={'geometry': 'school_geometry'}, inplace=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/3682636011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'school_geometry'}, inplace=True)


In [102]:
supermarket = gpd.read_file('../data/external_SA2/Supermarket.geojson')
supermarket_SA2 = gpd.sjoin(supermarket, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [103]:
property_agg5 = pd.merge(merged, supermarket_SA2, on='SA2_CODE21', how='left')

In [104]:
merged = property_agg5[['postcode', 'SA2_CODE21','SA2_district', 'property_geometry', 'station_geometry', 'park_geometry', 'shop_geometry','hospital_geometry','school_geometry', 'geometry']]
merged.rename(columns={'geometry': 'supermarket_geometry'}, inplace=True)

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_81663/4112485194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'supermarket_geometry'}, inplace=True)


In [105]:
merged.to_csv("../data/raw/aggregated_locations.csv")